# Change Log

Version 1: augmentations, added UNET model with SeResNext-50_32x4d encoder, predictions visualization <br>
Version 2: added predictions by one slide, padded and resized images/masks concatenation <br>
Version 5: added inference loop and submission <br>

This kernel is based on my previous one [HuBMAP: train/test patches generation](https://www.kaggle.com/mariazorkaltseva/hubmap-train-test-patches-generation)

**Configuration:**

Tile size: 256 without overlaping, contain artifacts

reduce rate: 4

Batch_size: 16 on train/val set, 1 on test set

Augmentations: Yes

Model: UNET architecture with SeResNext-50_32x4d encoder

Optimizer: Adam with lr=0.0001

Scoring function: Dice loss

Metric: IoU with 0.5 threshold


In [ ]:
!pip install segmentation_models_pytorch

In [ ]:
import os
import gc
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile as tiff
import cv2 as cv
import albumentations as albu

from tqdm.notebook import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
import segmentation_models_pytorch as smp
        
%matplotlib inline

In [ ]:
TRAIN_DATA_DIR = '../input/hubmap-256-tiles/train_tiles_256/train_tiles'
TEST_DATA_DIR = '../input/hubmap-256-tiles/test_tiles_256/test_tiles'
MODEL_SAVE_DIR = "/kaggle/working/"
TILE_SIZE = 256
REDUCE_RATE = 4
SEED = 42
BATCH_SIZE = 16
NUM_EPOCHS = 20

torch.cuda.empty_cache()


random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Helpers

In [ ]:
def display_pil_images(
    images, 
    masks=None,
    labels=None,
    columns=5, width=20, height=8, max_images=15, 
    label_wrap_length=50, label_font_size=9):

    if len(images) > max_images:
        print(f"Showing {max_images} images of {len(images)}:")
        images=images[0:max_images]
        if masks is not None:
            masks= masks[0:max_images]

    height = max(height, int(len(images)/columns) * height)
    plt.figure(figsize=(width, height))
    
    if masks is not None:
        for i, (image, mask) in enumerate(zip(images,masks)):
            plt.subplot(len(images) / columns + 1, columns, i + 1)
            plt.imshow(image)
            plt.imshow(mask, cmap='coolwarm', alpha=0.5)
            
            if labels is not None:
                plt.title(labels[i], fontsize=label_font_size); 
            
    else:
        for i, image in enumerate(images):
            plt.subplot(len(images) / columns + 1, columns, i + 1)
            plt.imshow(image)
        
            if labels is not None:
                plt.title(labels[i], fontsize=label_font_size);
    

def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
#with transposed mask
def rle_encode_less_memory(img):
    #the image should be transposed
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

In [ ]:
sample_df = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')
sample_df

# Image paths generation

In [ ]:
train_img_paths, train_mask_paths = [], []
for item1 in os.listdir(TRAIN_DATA_DIR):
    for item2 in os.listdir(os.path.join(TRAIN_DATA_DIR, item1)):
        if item2.startswith('img_'):
            train_img_paths.append(os.path.join(TRAIN_DATA_DIR, item1, item2))
        else:
            train_mask_paths.append(os.path.join(TRAIN_DATA_DIR, item1, item2))
            

test_img_paths = []
for item1 in os.listdir(TEST_DATA_DIR):
    for item2 in os.listdir(os.path.join(TEST_DATA_DIR, item1)):
        test_img_paths.append(os.path.join(TEST_DATA_DIR, item1, item2))

In [ ]:
def sort_by(x):
    return x.rsplit('/', 1)[1]

train_img_paths = sorted(train_img_paths, key = sort_by)
train_mask_paths = sorted(train_mask_paths, key = sort_by)
test_img_paths = sorted(test_img_paths, key = sort_by)
        
print(train_img_paths[:5])
print(train_mask_paths[:5])
print(test_img_paths[:5])

In [ ]:
print(f"Amount of samples in the train set {len(train_img_paths)}")
print(f"Amount of samples in the test set {len(test_img_paths)}")

# View images and masks

In [ ]:
imgs = [Image.open(img_path) for img_path in train_img_paths]
masks = [Image.open(mask_path) for mask_path in train_mask_paths]
display_pil_images(imgs[300:500], masks[300:500])

In [ ]:
imgs = [Image.open(img_path) for img_path in test_img_paths]
display_pil_images(imgs[450:500])

# Dataset class

In [ ]:
class HuBMAPDataset(D.Dataset):
    
    def __init__(
            self, 
            paths, 
            mode,
            augmentation=None,
            preprocessing=None,
    ):

        self.paths = paths
        self.mode = mode
        self.augmentation = augmentation
        self.preprocessing = preprocessing
        
    def __getitem__(self, i):       
        if self.mode in ['train', 'val']:
            image = cv.imread(self.paths[i][0])
            image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
            mask = cv.imread(self.paths[i][1], 0)
            mask = np.expand_dims(mask, axis=2)
        else:
            image = cv.imread(self.paths[i])
            image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        if self.augmentation:
            if self.mode in ['train', 'val']:
                sample = self.augmentation(image=image, mask=mask)
                image, mask = sample['image'], sample['mask']
            else:
                sample = self.augmentation(image=image)
                image = sample['image']

        if self.preprocessing:
            if self.mode in ['train', 'val']:
                sample = self.preprocessing(image=image, mask=mask)
                image, mask = sample['image'], sample['mask']
            else:
                sample = self.preprocessing(image=image)
                image = sample['image']

        if self.mode in ['train', 'val']:
            return image, mask
        
        return image
        
    def __len__(self):
        return len(self.paths)

# Augmentations

In [ ]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),
        albu.RandomRotate90(p=0.5),
        albu.Transpose(p=0.5),
        
        albu.ShiftScaleRotate(scale_limit=0.2, rotate_limit=0, shift_limit=0.2, p=0.2, border_mode=0),

        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
        
        albu.Compose([
            albu.VerticalFlip(p=0.5),              
            albu.RandomRotate90(p=0.5)]
        )
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    test_transform = [
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing():
    _transform = [
        albu.Normalize(mean=(0.65459856,0.48386562,0.69428385), 
                       std=(0.15167958,0.23584107,0.13146145), 
                       max_pixel_value=255.0, always_apply=True, p=1.0),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [ ]:
train_paths, val_paths = train_test_split(list(zip(train_img_paths, train_mask_paths)),
                                        test_size=0.2,  
                                        random_state=SEED,
                                        shuffle=True)

print("Amount of train samples:", len(train_paths))
print("Amount of val samples:", len(val_paths))

In [ ]:
augmented_dataset = HuBMAPDataset(train_paths,
                                  'train',
                                  augmentation=get_training_augmentation(),
                                )

for i in range(20):
    image, mask = augmented_dataset[1]
    visualize(image=image, mask=mask.squeeze(-1))

# Model

In [ ]:
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' 

model = smp.Unet(encoder_name=ENCODER, encoder_weights=ENCODER_WEIGHTS, activation=ACTIVATION)

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

# Train/val dataloaders

In [ ]:
train_dataset = HuBMAPDataset(train_paths, 
                              'train',
                              augmentation=get_training_augmentation(), 
                              preprocessing=get_preprocessing()
                            )

valid_dataset = HuBMAPDataset(val_paths,
                              'val',
                              augmentation=get_validation_augmentation(), 
                              preprocessing=get_preprocessing()
                            )

train_loader = D.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = D.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

# Train loop

In [ ]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
best_loss = 1.0

train_losses, val_losses = [], []
train_scores, val_scores = [], []

for i in range(0, NUM_EPOCHS):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    train_losses.append(train_logs['dice_loss'])
    val_losses.append(valid_logs['dice_loss'])
    train_scores.append(train_logs['iou_score'])
    val_scores.append(valid_logs['iou_score'])
    
    if best_loss > valid_logs['dice_loss']:
        best_loss = valid_logs['dice_loss']
        torch.save(model, os.path.join(MODEL_SAVE_DIR, 'best_model.pth'))
        print('Model saved!')

In [ ]:
plt.figure()
plt.plot(train_losses, label='train loss')
plt.plot(val_losses, label='val loss')
plt.legend()
plt.show();

In [ ]:
plt.figure()
plt.plot(train_scores, label='train score')
plt.plot(val_scores, label='val score')
plt.legend()
plt.show();

# Results visualization

In [ ]:
best_model = torch.load(os.path.join(MODEL_SAVE_DIR, 'best_model.pth'), map_location=torch.device(DEVICE))

In [ ]:
test_dataset = HuBMAPDataset(test_img_paths, 
                             'test',
                             preprocessing=get_preprocessing()
                        )

test_dataloader = D.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

test_dataset_vis = HuBMAPDataset(test_img_paths, 'test')

best_model.eval()

for i in range(30):
    n = np.random.choice(len(test_dataset))
    
    image_vis = test_dataset_vis[n].astype('uint8')
    image = test_dataset[n]
    
    x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
    with torch.set_grad_enabled(False):
        pr_mask = best_model.predict(x_tensor)
        pr_mask = (pr_mask.squeeze().cpu().numpy().round().astype('uint8'))
        
    visualize(
        image=image_vis, 
        predicted_mask=pr_mask
    )

# Сombining prediction masks

In [ ]:
def extract_slide_tiles(file_paths, id):
    def sort_by(x):
        # extract tile numbers
        return int(x.rsplit('/', 1)[1].split('_')[1].split('.')[0])
    return sorted(list(filter(lambda x: id in x, file_paths)), key=sort_by)

IDX = 'b2dc8411c'
slide_paths = extract_slide_tiles(test_img_paths, IDX)

# padded and reduced shape of corresponding image
height, width = (3840, 7936)

In [ ]:
# one slide predictions
test_dataset = HuBMAPDataset(slide_paths, 
                             'test',
                             preprocessing=get_preprocessing()
                        )

test_dataloader = D.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

test_dataset_vis = HuBMAPDataset(slide_paths, 'test')

best_model.eval()

mask_preds = []
for i, image in enumerate(tqdm(test_dataloader)):
    image_vis = test_dataset_vis[i].astype('uint8')
    
    image = image.to(DEVICE)
    with torch.set_grad_enabled(False):
        mask_pred = best_model(image)
        mask_pred = mask_pred.squeeze().cpu().numpy().round().astype('uint8')
    mask_preds.append(np.expand_dims(mask_pred, axis=0))
    
    if i % 20 == 0:
        visualize(
            image=image_vis, 
            predicted_mask=mask_pred
        )

mask_preds = np.concatenate(mask_preds)

In [ ]:
merge_image = np.zeros((height, width, 3))

k = 0
for i in range(0, height // TILE_SIZE):
    for j in range(0, width // TILE_SIZE):
        image = cv.imread(slide_paths[k])
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        merge_image[i*TILE_SIZE:i*TILE_SIZE + TILE_SIZE, j*TILE_SIZE:j*TILE_SIZE + TILE_SIZE, :] = image
        k += 1

In [ ]:
merge_mask = np.zeros((height, width))

k = 0
for i in range(0, height // TILE_SIZE):
    for j in range(0, width // TILE_SIZE):
        merge_mask[i*TILE_SIZE:i*TILE_SIZE + TILE_SIZE, j*TILE_SIZE:j*TILE_SIZE + TILE_SIZE] = mask_preds[k]
        k += 1

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(merge_image.astype('uint8'))
plt.imshow(merge_mask.astype('uint8'), cmap='coolwarm', alpha=0.5);

# Inference loop

In [ ]:
imgs_size_df = pd.read_csv("../input/hubmap-img-sizes/imgs_size.csv")
imgs_size_df.set_index(imgs_size_df['id'], inplace=True)

best_model.eval()

rle_preds = []
for idx in sample_df['id']:
    print(idx)
    
    slide_paths = extract_slide_tiles(test_img_paths, idx)
    
    test_dataset = HuBMAPDataset(slide_paths, 
                                 'test',
                                 preprocessing=get_preprocessing()
                                )

    test_dataloader = D.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

    mask_preds = []
    for i, image in enumerate(tqdm(test_dataloader)):

        image = image.to(DEVICE)
        with torch.set_grad_enabled(False):
            mask_pred = best_model(image)
            mask_pred = mask_pred.squeeze().cpu().numpy().round().astype('uint8')
        mask_preds.append(np.expand_dims(mask_pred, axis=0))

    mask_preds = np.concatenate(mask_preds)
    
    original_shape = imgs_size_df['orig_height'].loc[idx], imgs_size_df['orig_width'].loc[idx]
    height, width = imgs_size_df['reduced_height'].loc[idx], imgs_size_df['reduced_width'].loc[idx]
    
    merge_mask = np.zeros((height, width), dtype=np.uint8)

    k = 0
    for i in range(0, height // TILE_SIZE):
        for j in range(0, width // TILE_SIZE):
            merge_mask[i*TILE_SIZE:i*TILE_SIZE + TILE_SIZE, j*TILE_SIZE:j*TILE_SIZE + TILE_SIZE] = mask_preds[k]
            k += 1

    pad0 = (REDUCE_RATE*TILE_SIZE - original_shape[0]%(REDUCE_RATE*TILE_SIZE))%(REDUCE_RATE*TILE_SIZE)
    pad1 = (REDUCE_RATE*TILE_SIZE - original_shape[1]%(REDUCE_RATE*TILE_SIZE))%(REDUCE_RATE*TILE_SIZE)
    
    merge_mask = cv.resize(merge_mask,(merge_mask.shape[1]*REDUCE_RATE, merge_mask.shape[0]*REDUCE_RATE), 
                    interpolation = cv.INTER_AREA)

    merge_mask = merge_mask[pad0//2:-(pad0-pad0//2) if pad0>0 else merge_mask.shape[0], \
                            pad1//2:-(pad1-pad1//2) if pad1>0 else merge_mask.shape[1]]
        
    rle_preds.append(rle_encode_less_memory(merge_mask))
    
    del slide_paths, test_dataset, test_dataloader, mask_preds, merge_mask
    gc.collect()

In [ ]:
sample_df['predicted'] = rle_preds
sample_df.to_csv('submission.csv')